# __Rangement d'un tuyau de jardin__

Auguste Pierfitte et Cécile-Aurore Wotawa

On commence par importer les modules nécessaires. Nous avons choisi le module d'optimisation CasADi.

## Prise en main du problème

In [1]:
from casadi import *
import time
import numpy as np

In [2]:
N =
L = 
dx = L/N
gamma = 
K = 

SyntaxError: invalid syntax (<ipython-input-2-a3241307abc4>, line 1)

In [5]:
opti = casadi.Opti();
z = opti.variable(2, N+1)
f = K*(z[0,-1] - L/2)**2
for i in range(N+1):
    f += (dx*z[1,i])**2

NameError: name 'N' is not defined

In [4]:
opti.minimize(f)

for i in range(N+1):
    opti.subject_to((z[0,i+1]-z[0,i])**2 + (z[1,i+1]-z[1,i])**2 - dx**2 == 0)
    opti.subject_to(z[0,i]-z[0,i+1] <= 0)
    if i != 0:
        opti.subject_to((z[0,i+1]-2*z[0,i]+z[0,i-1])**2+(z[1,i+1]-2*z[1,i]+z[1,i-1])**2-(dx**4)*(gamma**2) <= 0)

opti.subject_to(z[1,-1] == 0)
opti.subject_to(z[0,0] == 0)
opti.subject_to(z[1,0] == 0)
opti.subject_to(z[1,1] >= 0)
        
# condition initiale : le tuyau d'arrosage est déplié sur toute sa longueur
z0 = np.array([np.linspace(0, L, N+1), np.zeros(N+1)])
opti.set_initial(z, z0)

NameError: name 'f' is not defined

## Réponse aux questions

### 1. Étude du problème d'optimisation

#### __Question 1 :__

Minimiser le premier terme du coût (4) permet de rapprocher la fin du tuyau (abscisse $x_N$) le plus proche possible du parterre de fleurs.

Minimiser le deuxième terme permet d'éviter que les ordonnées $y_i$ s'éloignent trop de l'axe $y=0$. Ainsi, le tuyau prend moins de place dans le jardin !

#### Question 2 :

**TODO**

#### Question 3 :

Cette condition impose que le tuyau d'arrosage ne "plie" pas trop.

En effet, elle impose qu'on puisse "insérer une courbe de degré 2 et de coefficient $\gamma$" entre 3 points successifs.


#### Question 4 :

- La variable de décision $z$ est un vecteur de $\mathbb{R}^{2(N + 1)}$, représentant les N+1 points $(x_i, y_i)$ : $z = ((x_0, y_0), ..., (x_N, y_N))$
- Il y a N+1 couples de variables de la forme $(x_i, y_i)$
- Pour les contraintes $c_{eq}$ :
    - $c_{eq_1}^i(z) = (x_{i+1} - x_i)^2 + (y_{i+1} - y_i)^2 - dx^2$, pour $0 \leq i \leq N-1$
    - $c_{eq_2}(z) = x_0$
    - $c_{eq_3}(z) = y_0$
    - $c_{eq_4}(z) = y_N$
    
Ce qui invite à poser $c_{eq}(z) = (c_{eq_1}^0(z), ..., c_{eq_1}^{N-1}(z), c_{eq_2}(z), c_{eq_3}(z), c_{eq_4}(z))$

- Pour les contraintes $c_{ineq}$ :
    - $ c_{ineq_1}^i(z) = x_i - x_{i+1}$, pour $0 \leq i \leq N-1$
    - $c_{ineq_2}^i(z) = (x_{i+1} - 2x_i + x_{i-1})^2 + (y_{i+1} - 2y_i + y_{i-1})^2 - dx^4\gamma^2$, pour $1 \leq i \leq N-1$
    - $c_{ineq_3}(z) = -y_1$
    
Ce qui invite à poser $c_{ineq}(z) = (c_{ineq_1}^0(z), ..., c_{ineq_1}^{N-1}(z), c_{ineq_2}^1(z), ..., c_{ineq_2}^{N-1}(z), c_{ineq_3}(z))$

- Finalement, la fonction objectif $f$ à minimiser est la suivante :
$f(z) = K(x_N - \frac{L}{2})^2 + dx^2\sum_{i=0}^{N}y_i^2$

Problème de contrôle optimal... A bientôt et bon courage !